In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('GSPC_2015_2020.csv')

In [7]:
spx = df.set_index('Date')['Adj Close']
spx

Date
2015-08-24    1893.209961
2015-08-25    1867.609985
2015-08-26    1940.510010
2015-08-27    1987.660034
2015-08-28    1988.869995
                 ...     
2020-08-17    3381.989990
2020-08-18    3389.780029
2020-08-19    3374.850098
2020-08-20    3385.510010
2020-08-21    3397.159912
Name: Adj Close, Length: 1259, dtype: float64

In [38]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
data_gen_df = spx/1000
data_gen = TimeseriesGenerator(data_gen_df.values, data_gen_df.diff().fillna(0), length=30)
data_gen[0][0].shape

(128, 30)

In [39]:
# https://github.com/usernaamee/keras-wavenet/blob/master/simple-generative-model.py

In [40]:
from tensorflow.keras.layers import Convolution1D, Convolution2D, Flatten, Dense, \
    Input, Lambda, Activation, BatchNormalization
from tensorflow import keras
import tensorflow

def WaveNetBlock(filters, kernel_size, dilation_rate, activation=None):
    def f(input_):
        residual = input_
        tanh_out = Convolution1D(filters, kernel_size,
                                       dilation_rate=dilation_rate,
                                       padding='same',
                                       activation='tanh')(input_)
        sigmoid_out = Convolution1D(filters, kernel_size,
                                          dilation_rate=dilation_rate,
                                          padding='same',
                                          activation='sigmoid')(input_)
        merged = keras.layers.Multiply()([tanh_out, sigmoid_out])
        skip_out = Convolution1D(1, 1, activation=activation, padding='same')(merged)
        out = keras.layers.Add()([skip_out, residual])
#         return out, skip_out
        return skip_out
    return f

In [41]:
x = i = Input(shape=(30, 1))
x = BatchNormalization()(x)
x = WaveNetBlock(filters=8, kernel_size=32, dilation_rate=1, activation='relu')(x)
x = WaveNetBlock(filters=4, kernel_size=4, dilation_rate=2, activation='relu')(x)
x = WaveNetBlock(filters=2, kernel_size=4, dilation_rate=4, activation='relu')(x)
# x = WaveNetBlock(filters=1, kernel_size=4, dilation_rate=4, activation='relu')(x)
x = Flatten()(x)
x = Dense(2, activation='relu')(x)
x = Dense(1, activation='relu')(x)

model = keras.Model(i, x)
model.compile(optimizer=tensorflow.keras.optimizers.Adam(10), loss='mse')
model.fit(data_gen, epochs=12)

Epoch 1/12
10/10 [==============================] - 0s 18ms/step - loss: 0.0010
Epoch 2/12
10/10 [==============================] - 0s 19ms/step - loss: 0.0010
Epoch 3/12
10/10 [==============================] - 0s 17ms/step - loss: 0.0010
Epoch 4/12
10/10 [==============================] - 0s 16ms/step - loss: 0.0010
Epoch 5/12
10/10 [==============================] - 0s 14ms/step - loss: 0.0010
Epoch 6/12
10/10 [==============================] - 0s 15ms/step - loss: 0.0010
Epoch 7/12
10/10 [==============================] - 0s 15ms/step - loss: 0.0010
Epoch 8/12
10/10 [==============================] - 0s 14ms/step - loss: 0.0010
Epoch 9/12
10/10 [==============================] - 0s 13ms/step - loss: 0.0010
Epoch 10/12
10/10 [==============================] - 0s 13ms/step - loss: 0.0010
Epoch 11/12
10/10 [==============================] - 0s 12ms/step - loss: 0.0010
Epoch 12/12
10/10 [==============================] - 0s 12ms/step - loss: 0.0010


In [42]:
model.predict(data_gen)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
data_gen_df['close'].diff().sum()

In [36]:
import tensorflow as tf
tf.config.experimental.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:XLA_GPU:0', device_type='XLA_GPU')]

In [1]:
0

0